<a href="https://colab.research.google.com/github/seanjudelyons/GPT3.5_Python/blob/main/GPT3_CustomContext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# enter your openai api key below
openai.api_key = "ENTER YOUR API KEY HERE"

# now enter a general question below
question = "what is a prevalent gas surrounding mars?"

In [ ]:
!pip install openai
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 87.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=1f3a91a69228e711d5067f7941dc4cc8a8a890f644a32d284c45fffef8886cbc
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
import openai
import csv
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer

In [ ]:
# create a knoledge base.  I am uploading the popular jeopardy dataset to create a context for general question answering (https://www.kaggle.com/datasets/tunguz/200000-jeopardy-questions?resource=download)
from google.colab import files

# Upload a file
uploaded = files.upload()

# Print the name of the uploaded file
for name, data in uploaded.items():
  print(f'Uploaded file "{name}" with length {len(data)} bytes')


Saving JEOPARDY_CSV.csv to JEOPARDY_CSV.csv
Uploaded file "JEOPARDY_CSV.csv" with length 34854057 bytes


In [ ]:
# sample question and answers from custom dataset
with open('JEOPARDY_CSV.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)

    data = []
    for row in csv_reader:
        data.append(row)

print(data[0].get(" Question"))
print(data[0].get(" Answer"))

In [ ]:
'''

This is not a necessery step but I am splitting off 99% of the data and only using 1% of the data. I am doing this to save time.

'''
# split the data into training and test sets
train_data, test_data = train_test_split(data, test_size=0.99, random_state=42)

# srint the first question and answer in the training set
print(train_data[0].get(" Question"))
print(train_data[0].get(" Answer"))

Contrapuntal is the adjectival form of this, the combination of melodic lines
counterpoint


In [ ]:
# load the "text-embedding-ada-002" model
def create_embeddings(prompt):
    response = openai.Embed.create(
        model="text-embedding-ada-002",
        prompt=prompt
    )
    return np.array(response['embeddings'])

# load model to create embeddings
embed_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# create embeddings for our questions and answers dataset
data_embeddings = embed_model.encode([row[" Question"] for row in train_data])

In [ ]:
# embedding the question
question_embedding = embed_model.encode(question)

In [ ]:
# calculate cosine similarity
cosine_similarities = cosine_similarity(question_embedding.reshape(1, -1), data_embeddings)

The cosine similarity between the embeddings:

$$
\text{cosine similarity}(A, B) = \frac{A \cdot B}{\|A\| \|B\|}
$$




In [ ]:
# sort questions based on cosine similarity scores
sorted_indices = np.argsort(cosine_similarities[0])[::-1]

# select the top 10 questions and their corresponding answers
top_10_questions = [train_data[i][" Question"] for i in sorted_indices[:10]]
top_10_answers = [train_data[i][" Answer"] for i in sorted_indices[:10]]

In [ ]:
# Create a context using the top 10 questions and answers
context = "\n".join([f"Q: {q}\nA: {a}" for q, a in zip(top_10_questions, top_10_answers)])

# Use the context with "text-davinci-003"
prompt = f"{context}\n\nQ: {question}\nA:"
response = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    max_tokens=100,
    n=1,
    stop=None,
    temperature=0.5,
)

# Print the context
print("Context:")
print(context)
print()

# Print the response
print("Answer:")
print(response.choices[0].text.strip())


Context:
Q: If we were on Mars, the atmosphere would be incredibly thin, consisting mainly of this gas
A: carbon dioxide
Q: In November 2009 science news, a NASA probe found water here, & not a little bit, either
A: the Moon
Q: It's the alliterative term for coal, oil & natural gas made from the remains of living organisms
A: fossil fuels
Q: 6 letters, score 24: Composed of silicon dioxide, it's the most abundant mineral on Earth's surface
A: quartz
Q: Layers of sulfuric acid clouds completely obscure the surface of this neighboring planet
A: Venus
Q: 1 of 2 elements more abundant than aluminum in the Earth's crust
A: silicon (or oxygen)
Q: The splitting of a heavy nucleus -- it sounds like where Fermi has "gone" on vacation
A: fission
Q: A recent study found Gliocladium roseum, a parasitic one of these organisms, can produce a gas termed myco-diesel
A: a fungus
Q: In 1971 Mariner 9 discovered a volcano on this planet rising 15 1/2 miles above the surface
A: Mars
Q: Chemist Svante Arrh